In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import os

# --- 1. Carga de Datos de 5 Minutos ---
PROCESSED_DATA_PATH = "data_processed" 

df_train_val = pd.read_csv(
    os.path.join(PROCESSED_DATA_PATH, "df_cincoMinutos_train_val.csv"), 
    index_col='timestamp', 
    parse_dates=True
)
df_test_final = pd.read_csv(
    os.path.join(PROCESSED_DATA_PATH, "df_cincoMinutos_test_final.csv"), 
    index_col='timestamp', 
    parse_dates=True
)

# 2. Creación de la característica de Volatilidad
df_train_val['volatilidad'] = df_train_val['high'] - df_train_val['low']

# Si decides usar solo 'close' para univariado, ajusta la lista 'features' aquí:
#features = ['open', 'high', 'low', 'close', 'volume', 'volatilidad']
features = ['close']

# --- 2. Creación y Ajuste del Escalador de 5 Minutos ---
data_train_val = df_train_val[features].values 

scaler_minuto = MinMaxScaler(feature_range=(0, 1))
scaled_data_train_val = scaler_minuto.fit_transform(data_train_val)

print("--- Escalador de 5 Minutos Creado y Ajustado ---")
print(f"Forma del Array de entrenamiento escalado: {scaled_data_train_val.shape}")

# --- 3. Creación de Secuencias (Ventana de 3 Días) ---
# 3 días * 24 horas/día * 12 pasos de 5 min/hora = 864 pasos
TIMESTEP = 864 

X_train_minuto = []
Y_train_minuto = []
close_idx = features.index('close') # Índice de la columna 'close'

for i in range(TIMESTEP, len(scaled_data_train_val)):
    X_train_minuto.append(scaled_data_train_val[i-TIMESTEP:i, :]) 
    Y_train_minuto.append(scaled_data_train_val[i, close_idx])

X_train_minuto, Y_train_minuto = np.array(X_train_minuto), np.array(Y_train_minuto)

print("\n--- Secuencias de 5 Minutos Creadas ---")
print(f"Forma de X_train_minuto: {X_train_minuto.shape}")

--- Escalador de 5 Minutos Creado y Ajustado ---
Forma del Array de entrenamiento escalado: (526464, 1)

--- Secuencias de 5 Minutos Creadas ---
Forma de X_train_minuto: (525600, 864, 1)


In [7]:
    #Modelo LSTM:
    # Una capa recurrente con 50 unidades
    # Una capa de Dropout para prevenir el sobreajuste.
    # Una capa Densa (Dense) de salida con 1 unidad, 
    #       utilizando activación lineal, ya que es una tarea de regresión 
    #       (prediciendo un valor continuo: el precio).

    def crear_modelo_lstm(input_shape):
        """Define y compila el modelo LSTM."""
        
        # El input_shape debe ser (TIMESTEP, features) -> (60, 6)
        model = Sequential()
        
        # Capa LSTM
        model.add(LSTM(
            units=50, 
            return_sequences=False, # False porque solo nos interesa la salida del último paso de tiempo
            input_shape=input_shape
        ))
        
        # Capa de Dropout (para regularización)
        model.add(Dropout(0.2))
        
        # Capa de Salida (Predicción de 1 valor: Precio de Cierre)
        model.add(Dense(units=1))
        
        # Compilación: Usamos 'adam' como optimizador y 'mean_squared_error' (MSE) como métrica
        # MSE es común en tareas de regresión.
        model.compile(optimizer='adam', loss='mean_squared_error')
        
        return model

    #Modelo GRU

    def crear_modelo_gru(input_shape):
        """Define y compila el modelo GRU."""
        
        model = Sequential()
        
        # Capa GRU
        model.add(GRU(
            units=50, 
            return_sequences=False, 
            input_shape=input_shape
        ))
        
        # Capa de Dropout
        model.add(Dropout(0.2))
        
        # Capa de Salida
        model.add(Dense(units=1))
        
        # Compilación
        model.compile(optimizer='adam', loss='mean_squared_error')
        
        return model


In [6]:
# Configuración de Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

input_shape_minuto = (X_train_minuto.shape[1], X_train_minuto.shape[2]) 

# 1. Crear Modelos
modelo_lstm_minuto = crear_modelo_lstm(input_shape_minuto)
modelo_gru_minuto = crear_modelo_gru(input_shape_minuto)

# 2. Entrenamiento
print("\n--- Iniciando Entrenamiento LSTM Minuto ---")
history_lstm_minuto = modelo_lstm_minuto.fit(
    X_train_minuto, 
    Y_train_minuto, 
    epochs=25,        
    batch_size=256,    # Batch size alto para acelerar
    validation_split=0.2, 
    callbacks=[early_stopping], 
    verbose=1
)

print("\n--- Iniciando Entrenamiento GRU Minuto ---")
history_gru_minuto = modelo_gru_minuto.fit(
    X_train_minuto, 
    Y_train_minuto, 
    epochs=25, 
    batch_size=256, 
    validation_split=0.2, 
    callbacks=[early_stopping], 
    verbose=1
)

# 3. Guardado
modelo_lstm_minuto.save('modelo_lstm_cincoMinutos') 
modelo_gru_minuto.save('modelo_gru_cincoMinutos')   
joblib.dump(scaler_minuto, 'scaler_cincoMinutos.pkl')

print("\n✅ Modelos y Escalador de 5 Minutos Guardados con Éxito.")

C:\Users\crges\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



--- Iniciando Entrenamiento LSTM Minuto ---
Epoch 1/25
  19/1643 ━━━━━━━━━━━━━━━━━━━━ 24:25 903ms/step - loss: 0.0484

KeyboardInterrupt: 